In [2]:
import keras
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
import keract

In [3]:
# Helper functions to load data

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data, y_data

Define paths to the model and data files

In [6]:
model_path = 'models/sunglasses_bd_net.h5'
clean_data_path = 'data/clean_test_data.h5'
pois_data_path = 'data/sunglasses_poisoned_data.h5'
val_data_path = 'data/clean_validation_data.h5i

Load in data from the h5 files

In [13]:
bd_model = keras.models.load_model(model_path)
x_clean, y_clean = data_loader(clean_data_path)
x_pois, y_pois = data_loader(pois_data_path)
x_val, y_val = data_loader(val_data_path)

Now let's use keract to extract representations form the Badnet from the 2nd last layer

In [18]:
lname = 'add_1'
target = y_pois[0]

rep_clean = keract.get_activations(bd_model, x_clean[np.where(y_clean==target)], layer_names=lname, nodes_to_evaluate=None, output_format='simple', nested=False, auto_compile=True)[lname]
rep_pois = keract.get_activations(bd_model, x_pois, layer_names=lname, nodes_to_evaluate=None, output_format='simple', nested=False, auto_compile=True)[lname]
rep_val = keract.get_activations(bd_model, x_val[np.where(y_val==target)], layer_names=lname, nodes_to_evaluate=None, output_format='simple', nested=False, auto_compile=True)[lname]

We choose create a representation matrix M such that it captures the changes that the poisoning is trying to cause

In [20]:
M = rep_pois - rep_clean.mean(axis=0)